In [1]:
# default_exp scraper

# Scraper

> This module is used to scrape https://www.point2homes.com for all residential in Scottsdale Arizona

In [13]:
#hide
from nbdev.showdoc import *
from lxml.etree import HTML
from selenium import webdriver
from selenium.webdriver.common.by import By
import xmltodict as xd
from time import sleep
from lxml import etree
import pandas as pd
import numpy as np

In [2]:
def get_text(xpath):
    return re.sub('<[^<]+?>', '', etree.tostring(xpath, method='html', with_tail=False).decode('ascii')).replace('\n', '').replace('  ', '')

In [3]:
def extract_properties(land):
    data = xd.parse(etree.tostring(land))['div']['div']
    try:
        address = data[0]['@data-address']
    except:
        print("Missing Address")
        address = ''
    try:
        price = data[1]['@data-price']
        try:
            lot_size = data[2]['ul']['li'][0]['strong']
        except:
#             print("Missing lot_size")
            lot_size = ''
        try:
            beds = data[2]['ul']['li'][0]['strong']
        except:
#             print("missing beds")
            beds = ''
        try:
            baths = data[2]['ul']['li'][1]['strong']
        except:
#             print("missing baths")
            baths = ''
        try:
            sqft = data[2]['ul']['li'][2]['strong']
        except:
#             print("missing sqft")
            sqft = ''
        try:
            property_type = data[2]['ul']['li'][3]['#text']
        except:
#             print("Missing property_type")
            property_type = ''
        link = data[4]['a']['@href']
        properties = {
            'address': address,
            'price':price,
            'lot_size':lot_size,
            'beds':beds,
            'baths':baths,
            'sqft':sqft,
            'property_type':property_type,
            'link':link
        }
        return properties
    except Exception as e:
        raise e
        print(f'Failed on {address}')

In [4]:
def read_land_column(lands):
    land_list = []
    for land in lands:
        land_list.append(extract_properties(land))
    return land_list

In [5]:
url = "https://www.point2homes.com/CA/Real-Estate-Maps/BC/Vancouver-Island.html"
browser = webdriver.Firefox()
browser.get(url)

In [6]:
page_source = HTML(browser.page_source)

lands = page_source.xpath("//div[@class='item_information']")


In [ ]:
land_data = []


In [14]:

while(1):
    # Read Page
    page_source = HTML(browser.page_source)

    # Extract Land Column
    lands = page_source.xpath("//div[@class='item_information']")

    # Append data
    land_data = land_data + read_land_column(lands)
    
    # Report in.
    print(f"Successfully Scraped: {len(lands)}")

    # Go to next page
    next_button = browser.find_element(By.XPATH, "//a[starts-with(@class,'pager-next')]")
    if 'disabled' in next_button.get_attribute('class'):
        break
    else:
        print('Next Page')
        next_button.click()
        sleep(8)

Missing property_type
missing baths
missing sqft
Missing property_type
Missing property_type
Missing property_type
missing sqft
Missing property_type
Missing property_type
Missing property_type
Missing property_type
Missing property_type
Missing property_type
Missing property_type
Missing property_type
Missing property_type
Missing property_type
Missing property_type
Missing property_type
Missing lot_size
missing beds
missing baths
missing sqft
Missing property_type
Missing property_type
Missing property_type
Missing property_type
Missing property_type
Missing property_type
missing baths
missing sqft
Missing property_type
missing baths
missing sqft
Missing property_type
Missing property_type
Missing property_type
Missing property_type
Missing property_type
Missing lot_size
missing beds
missing baths
missing sqft
Missing property_type
missing sqft
Missing property_type
Missing property_type
missing sqft
Missing property_type
Missing property_type
Missing property_type
Missing property_t

In [15]:
df = pd.DataFrame(land_data)

In [4]:
import re

In [19]:
def extract_numbers(s):
    try:
        if s == np.nan:
            return 0
        if s == 'NaN':
            return 0
        return float(''.join(re.findall("[-+]?\d*\.\d+|\d+", s)) or 0)
    except:
        return 0

In [6]:
df['price_float'] = df['price'].apply(extract_numbers)

In [7]:
df['sqft_float'] = df['sqft'].apply(extract_numbers)

TypeError: expected string or bytes-like object

In [21]:
df['Dollar per Sqft'] = df['price_float'] / df['sqft_float']

In [22]:
df = df.sort_values('Dollar per Sqft')

In [23]:
df['link'] = 'https://www.point2homes.com' + df['link']

In [24]:
df['Dollar per Sqft'] = df['Dollar per Sqft'].apply(lambda x: str(round(x, 2)))

In [28]:
df = df.drop_duplicates('address')

In [58]:
df = pd.read_csv('vanland_homes.csv')

In [20]:
df['baths'] = df['baths'].apply(extract_numbers)

In [26]:
df['beds'] = df['beds'].apply(extract_numbers)

In [38]:
homes = df[(df['baths'] >= 3) & (df['beds'] >= 4)].drop(['lot_size', 'property_type'], axis=1).sort_values('price_float')

In [45]:
homes = homes.sort_values('Dollar per Sqft')
homes

,address,price,beds,baths,sqft,link,price_float,sqft_float,Dollar per Sqft
10,851 2nd Avenue,"$229,000 CAD",6.0,4.0,"4,608",https://www.point2homes.com/CA/Home-For-Sale/B...,229000.0,4608.0,49.70
48,238 Murtle Cres,"$275,000 CAD",5.0,4.0,"2,392",https://www.point2homes.com/CA/Home-For-Sale/B...,275000.0,2392.0,114.97
60,4748 NICHOLSON RD,"$289,000 CAD",4.0,3.0,"2,300",https://www.point2homes.com/CA/Home-For-Sale/B...,289000.0,2300.0,125.65
70,4550 COPPER RIVER ROAD,"$499,000 CAD",6.0,3.0,"3,764",https://www.point2homes.com/CA/Home-For-Sale/B...,499000.0,3764.0,132.57
74,7683 ST MARK CRESCENT,"$359,900 CAD",6.0,3.0,"2,680",https://www.point2homes.com/CA/Home-For-Sale/B...,359900.0,2680.0,134.29
...,...,...,...,...,...,...,...,...,...
2288,2957 LITTLE BIG BAR LAKE,"$650,000 CAD",2973.0,95.0,NaN,https://www.point2homes.com/CA/Home-For-Sale/B...,650000.0,0.0,inf
2431,2098 Walsh Road,"$645,000 CAD",4.0,3.0,NaN,https://www.point2homes.com/CA/Home-For-Sale/B...,645000.0,0.0,inf
2363,8932 65 Avenue,"$369,900 CAD",6.0,3.0,NaN,https://www.point2homes.com/CA/Home-For-Sale/A...,369900.0,0.0,inf
2428,9695 287 STREET,"$1,400,000 CAD",21736440.0,21736440.0,NaN,https://www.point2homes.com/CA/Home-For-Sale/B...,1400000.0,0.0,inf


In [66]:
homes['Region'] = homes['link'].apply(lambda x: x.split('/')[6])
homes['Community'] = homes['link'].apply(lambda x: x.split('/')[7])

In [67]:
homes[homes['address'] == '9902 Echo Heights ']

,address,price,beds,baths,sqft,link,price_float,sqft_float,Dollar per Sqft,Region,Community
243,9902 Echo Heights,"$545,979 CAD",4.0,3.0,"2,753",https://www.point2homes.com/CA/Home-For-Sale/B...,545979.0,2753.0,198.32,Vancouver-Island,Cowichan-Valley


In [68]:
vanland_homes = homes[homes['Region'] == 'Vancouver-Island']
vanland_homes.to_csv('vanland_homes.csv', index=False)

In [69]:
vanland_homes.sort_values('price_float').to_csv('vanland_homes_cheapest.csv', index=False)

In [71]:
vanland_homes.to_csv('vanland_homes.csv')

In [ ]:
len(pages)extract_numbers

121

In [ ]:
df = pd.DataFrame(land_list)

In [ ]:
df.sort_values('price_float').to_csv('VancouverIsland-Cheapest.csv',index=False)

In [ ]:
df.iloc[92].values

array(['LT 38 RUXTON ISLAND ', '$75,000 CAD', '2.15 ac', 'Land',
       '/CA/Vacant-Land-For-Sale/BC/Vancouver-Island/Cowichan-Valley/LT-38-RUXTON-ISLAND/88932209.html'],
      dtype=object)

In [ ]:
next_page = browser.find_element(By.XPATH, "//input[starts-with(@name,'username')]")
password = browser.find_element(By.XPATH, "//input[starts-with(@name,'password')]")

username.send_keys("Carole@impactisi.com")
password.send_keys("password123")
browser.find_element(By.XPATH, "//input[starts-with(@name,'Login')]").click()

In [ ]:
pager-next

In [ ]:
# price, ac, price/ac, datetime, id, area

In [ ]:
1038 / 100

10.38

In [ ]:
item = {
    'address':
    'price':
    'characteristics':
    'labels':
    'detail_link'
}

SyntaxError: invalid syntax (<ipython-input-111-5589d66c2c9c>, line 3)

In [ ]:
import pandas as pd

In [ ]:
from lxml import etree

In [ ]:
etree.tostring(tree)

b'<div class="item_information">\n        <div class="item_address" data-url="/CA/Vacant-Land-For-Sale/BC/Nanoose-Bay/LOT-1-Oak-Leaf-Drive/82258309.html" data-tracking="" data-address="LOT 1 Oak Leaf Drive ">\n            <div class="address-container">\n                <span>LOT 1 Oak Leaf Drive </span>\n            </div>\n\n        </div>\n<div class="price " data-price="$799,000 CAD">\n    <span class="green">\n            <span> $799,000 CAD</span>\n    </span>\n\n</div>\n        <div class="characteristics-cnt">\n\n\n<ul>\n                    <li data-label="Lot Size" class="ic-lotsize">\n            <strong>0.842 ac</strong> <span class="gray">Lot Size</span>\n        </li>\n            <li class="property-type ic-proptype">\n            Land\n        </li>\n</ul>\n\n\n        </div>\n        <div class="labels">\n            <div class="featured-icon" title="Featured"/>\n\n        </div>\n        <div class="lnk_details">\n            <a rel="" href="/CA/Vacant-Land-For-Sale/BC

In [ ]:
etree.tostringlist(tree)

[b'<div class="item_information">\n        <div class="item_address" data-url="/CA/Vacant-Land-For-Sale/BC/Nanoose-Bay/LOT-1-Oak-Leaf-Drive/82258309.html" data-tracking="" data-address="LOT 1 Oak Leaf Drive ">\n            <div class="address-container">\n                <span>LOT 1 Oak Leaf Drive </span>\n            </div>\n\n        </div>\n<div class="price " data-price="$799,000 CAD">\n    <span class="green">\n            <span> $799,000 CAD</span>\n    </span>\n\n</div>\n        <div class="characteristics-cnt">\n\n\n<ul>\n                    <li data-label="Lot Size" class="ic-lotsize">\n            <strong>0.842 ac</strong> <span class="gray">Lot Size</span>\n        </li>\n            <li class="property-type ic-proptype">\n            Land\n        </li>\n</ul>\n\n\n        </div>\n        <div class="labels">\n            <div class="featured-icon" title="Featured"/>\n\n        </div>\n        <div class="lnk_details">\n            <a rel="" href="/CA/Vacant-Land-For-Sale/B

In [ ]:
pd.read_html(tree)

/usr/lib/python3.6/urllib/parse.py:107: FutureWarning: The behavior of this method will change in future versions. Use specific 'len(elem)' or 'elem is not None' test instead.
  return tuple(x.decode(encoding, errors) if x else '' for x in args)


TypeError: Cannot read object of type '_Element'

In [ ]:
tree = lands[0]

In [ ]:
price = tree.xpath("//div[@class='price']")


In [ ]:
price

[<Element div at 0x7fb8ac045a88>]

In [ ]:
e = price[0]

In [ ]:
e.text

'$47,900 CAD'

In [ ]:
address = tree.xpath("//div[@class='item_address']")


In [ ]:
address[0].attrib['data-address']

'LOT 1 Oak Leaf Drive '

In [ ]:
data-address

In [ ]:
descriptions = tree.xpath("//div[@class='info member-info']")
links = tree.xpath("//a[@class='learn-more']")

In [ ]:
lands

[<Element div at 0x7fb8ac0d0408>,
 <Element div at 0x7fb8ac0d0d08>,
 <Element div at 0x7fb8ac0f57c8>,
 <Element div at 0x7fb8ac045448>,
 <Element div at 0x7fb8ac045488>,
 <Element div at 0x7fb8ac045508>,
 <Element div at 0x7fb896df03c8>,
 <Element div at 0x7fb896df0408>,
 <Element div at 0x7fb896df0448>,
 <Element div at 0x7fb8ac0454c8>,
 <Element div at 0x7fb896df0488>,
 <Element div at 0x7fb896df04c8>,
 <Element div at 0x7fb896df0508>,
 <Element div at 0x7fb896df0548>,
 <Element div at 0x7fb896df0588>,
 <Element div at 0x7fb896df05c8>,
 <Element div at 0x7fb896df0608>,
 <Element div at 0x7fb896df0648>,
 <Element div at 0x7fb896df0688>,
 <Element div at 0x7fb896df06c8>,
 <Element div at 0x7fb896df0708>,
 <Element div at 0x7fb896df0748>,
 <Element div at 0x7fb896df0788>,
 <Element div at 0x7fb896df07c8>,
 <Element div at 0x7fb896df0808>,
 <Element div at 0x7fb896df0848>,
 <Element div at 0x7fb896df0888>,
 <Element div at 0x7fb896df08c8>,
 <Element div at 0x7fb896df0908>,
 <Element div 